# 1. Libraries, Configuration, and Importing Queries

## 1.1 Libraries

In [184]:
# selenium specific imports
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException

# other imports
import configparser
import time
import pandas as pd
import numpy as np
from datetime import datetime

## 1.2 Configuration

In [185]:
# configuration parser initialization
config = configparser.ConfigParser()
config.read('../config.ini')
delay = 10 # waits for 10 seconds for the correct element to appeara

## 1.3 Load csv of Brand Names Search Queries

-  Brand queries in conjuction with slight modifications were systematically created by Catherine C. Pollack at Dartmouth College. 

In [186]:
query_df = pd.read_csv("../data/queries/Final_Words_List.csv")

In [187]:
query_df.describe()

,search_query
count,670
unique,670
top,RWKnudsen
freq,1


# 2. Custom Functions 

## 2.1 Login

In [188]:
def login_streamhatchet():
    driver.get("https://app.streamhatchet.com/")
    driver.find_element_by_id("cookiesAccepted").click()

    username = driver.find_element_by_name("loginEmail")
    username.clear()
    username.send_keys(config['login_credentials']['email'])

    password = driver.find_element_by_name("loginPassword")
    password.clear()
    password.send_keys(config['login_credentials']['password'])

    driver.find_element_by_xpath("//button[contains(text(),'Login')]").click()
    time.sleep(3) # sleep for 3 seconds to let the page load

## 2.1 Stream Title Search

In [189]:
def stream_title_search(query, incomplete_queries_list, df):
    driver.get("https://app.streamhatchet.com/search/toolstatus")
    time.sleep(1)
    
    # Enters query into 'Stream title query'
    stream_title_query_input = WebDriverWait(driver, delay).until(EC.element_to_be_clickable((By.XPATH,"//input[@id='status-query']")))
    stream_title_query_input.send_keys(query)

    # Makes twitch the only platform to search
    platform_input = WebDriverWait(driver, delay).until(EC.element_to_be_clickable((By.XPATH,"//input[@class='search']")))
    platform_input.click()
    platform_input.send_keys(Keys.BACKSPACE)
    platform_input.send_keys(Keys.BACKSPACE)
    platform_input.send_keys(Keys.BACKSPACE)

    # Click to Expand Date Options
    driver.find_element_by_xpath("//div[@id='NewRangePicker']").click()
    
    # change the hours and minutes to 0:00 for date from and to 
    driver.find_element_by_xpath("//div[@class='calendar left']//select[@class='hourselect']//option[1]").click()
    driver.find_element_by_xpath("//div[@class='calendar left']//option[contains(text(),'00')]").click()
    driver.find_element_by_xpath("//div[@class='calendar right']//select[@class='hourselect']//option[1]").click()
    driver.find_element_by_xpath("//div[@class='calendar right']//option[contains(text(),'00')]").click()
    
    # Keep clicking on right_arrow
    while driver.find_element_by_xpath("//i[@id='icon-down-New']").is_displayed() == True:
        try:
            driver.find_element_by_xpath("//i[@class='fa fa-chevron-right glyphicon glyphicon-chevron-right']").click()
        except:
            break
                
    
    # Click on first day of the month:
    driver.find_element_by_xpath("//div[@class='calendar left']//td[contains(text(), '1')]").click()
    time.sleep(5)
    driver.find_element_by_xpath("//div[@class='calendar right']//td[contains(text(), '1')]").click()
    time.sleep(10)

    # Runs the search
    driver.find_element_by_xpath("//button[@class='applyBtn btn btn-sm btn-success ui google plus button']").click()
    run_button = WebDriverWait(driver, delay).until(EC.element_to_be_clickable((By.XPATH,"//button[@class='medium ui google plus submit button']")))
    run_button.click()
    
    # Scrape the Number of Titles
    num_titles = WebDriverWait(driver, delay).until(EC.visibility_of_element_located((By.XPATH,"//p[@id='messages-count']")))
    num_titles = num_titles.text

    # create a row_dict and append it to the df
    row_dict = {
        'query': query,
        'month': "Maybe May",
        'num_titles':num_titles
    }

    df = df.append(row_dict, ignore_index = True)

        
    incomplete_queries_list.append(query)
        
    return(df)
    
        

# 3. Run Stream Titles Search

In [ ]:
df = pd.DataFrame(columns=['query', 'month', 'num_titles'])
incomplete_queries_list = []
driver = webdriver.Chrome()

login_streamhatchet()
stream_title_search("test", incomplete_queries_list, df)